**References**

[Tutorial: Working with Hugging Face Models and Datasets](https://github.com/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch4_Fine_Tuning_Pretrained_Model_for_Question_Answering.ipynb)

[Fine-Tuning-for-Question-Answering-SQuAD-IndoQA](https://github.com/PrasetyoWidyantoro/Fine-Tuning-for-Question-Answering-SQuAD-IndoQA/blob/master/fine-tune-squad-dataset.ipynb)

# Import libraries

In [1]:
%pip install evaluate sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from tqdm.auto import tqdm
import evaluate
import numpy as np
import pandas as pd
import os
import json
from sentence_transformers import SentenceTransformer, util

# Global variables

In [3]:
random_seed = 42

kaggle = False

model_name = "openai-community/gpt2-medium"
model_short_name = "gpt2"

dataset_name = ("xquad", "xquad.ro")
dataset_short_name = "roitd_aug"

# Import model and tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Split the dataset into train, validation and test

In [5]:
ds = load_dataset(dataset_name[0], dataset_name[1])

ds = ds['validation'].train_test_split(test_size=0.2)
ds_train = ds['train']
ds_test = ds['test']

ds_test = ds_test.train_test_split(test_size=0.5)
ds_val = ds_test['train']
ds_test = ds_test['test']

print(len(ds_train), len(ds_val), len(ds_test))

952 119 119


In [6]:
with open('/kaggle/input/roitdaug/ds_roitd_augumented.json' if kaggle else 'ds_roitd_augumented.json', 'r') as f:
    ds_roitd = json.load(f) 

ds_train = Dataset.from_pandas(pd.DataFrame(ds_roitd))
ds_train

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 4417
})

# Process the dataset

## Train dataset

In [7]:
def standardize(text: str):
    return text
    # return text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

def process_and_tokenize(dataset, tokenizer) -> dict:
    # Standardize and prepare inputs
    questions = [q.strip() for q in dataset["question"]]
    contexts = dataset["context"]
    answers = dataset["answers"]

    # Tokenize with overflow handling and offset mapping
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Create mappings
    sample_mapping = inputs["overflow_to_sample_mapping"]
    offset_mapping = inputs["offset_mapping"]

    # Initialize start and end positions
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Retrieve original sample index
        sample_idx = sample_mapping[i]

        # Get context answer data
        answer_texts = answers[sample_idx]["text"]
        answer_starts = answers[sample_idx]["answer_start"]

        # Choose the answer span to use
        if len(answer_starts) > 1:
            start_char = answer_starts[1]
            end_char = start_char + len(answer_texts[1])
        else:
            start_char = answer_starts[0]
            end_char = start_char + len(answer_texts[0])

        # Token-level sequence IDs
        sequence_ids = inputs.sequence_ids(i)

        # Context span
        context_start = sequence_ids.index(1)
        context_end = next(
            (j - 1 for j in range(context_start, len(sequence_ids)) if sequence_ids[j] != 1),
            len(sequence_ids) - 1  # Default to the last valid token index
        )
        # Check if answer falls outside the current span
        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            # Mark as no answer
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find the token positions corresponding to the character-level answer
            start_token = next(
                (j for j in range(context_start, context_end + 1) if offsets[j][0] <= start_char < offsets[j][1]),
                None,
            )
            end_token = next(
                (j for j in range(context_end, context_start - 1, -1) if offsets[j][0] < end_char <= offsets[j][1]),
                None,
            )

            if start_token is not None and end_token is not None:
                start_positions.append(start_token)
                end_positions.append(end_token)
            else:
                # Fallback if token positions are not found
                start_positions.append(0)
                end_positions.append(0)

    # Update tokenized inputs with labels
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    # Remove unnecessary fields
    inputs.pop("overflow_to_sample_mapping", None)
    inputs.pop("offset_mapping", None)

    return inputs


In [8]:
ds_train

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 4417
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token

ds_tok_train = ds_train.map(lambda x: process_and_tokenize(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_train.column_names)
# ds_tok_train = ds_train.map(lambda x: process_and_tokenize(dataset=ds_train, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_train.column_names)
print(len(ds_tok_train))
# ds_tok_train

Map:   0%|          | 0/4417 [00:00<?, ? examples/s]

4434


In [10]:
print(ds_tok_train)

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 4434
})


## Validation and test datasets

In [11]:
def process_and_tokenize_val_test(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_i = sample_mapping[i]
        example_ids.append(dataset["id"][sample_i])

        seq_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        
        for j, ofs in enumerate(offset):
            inputs['offset_mapping'][i][j] = ofs if seq_ids[j] == 1 else None

    inputs["example_id"] = example_ids
    
    inputs.pop("overflow_to_sample_mapping")
    
    return inputs

In [12]:
ds_tok_val = ds_val.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_val.column_names)
ds_tok_test = ds_test.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_test.column_names)
 
print(len(ds_tok_val), len(ds_tok_test))

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

187 189


# Make metrics

In [13]:
print(ds_tok_val)

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 187
})


In [14]:
%pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [15]:
squad_metric = evaluate.load("squad")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

In [16]:
def compute_metrics(start_logits: list, end_logits: list, tok: dict, dataset: Dataset, max_answer_length: int = 30, eval_answer: SentenceTransformer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")) -> dict:
    extracted_features = {}
    
    # Group extracted features by ids
    for i, feature in enumerate(tok):
        if feature["example_id"] not in extracted_features:
            extracted_features[feature["example_id"]] = [i]
        else:
            extracted_features[feature["example_id"]].append(i)
    
    golds_squad = [{"id": data["id"], "answers": data["answers"]} for data in dataset]
    preds_squad = []
    
    golds_bleu = []
    preds_bleu = []
    
    for data in tqdm(dataset):
        answers = []

        # Iterate over all the extracted features
        for i in extracted_features[data["id"]]:
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            offs = tok[i]["offset_mapping"]

            # Get all combinations of start and end positions
            for start_i in range(len(start_logit)):
                for end_i in range(len(end_logit)):
                    # Continue on wrong answers
                    if offs[start_i] is None \
                        or offs[end_i] is None \
                        or end_i < start_i \
                        or end_i - start_i + 1 > max_answer_length:
                        continue
                    
                    # Add text and score
                    answer = {
                        "answer": data["context"][offs[start_i][0] : offs[end_i][1]],
                        "score": start_logit[start_i] + end_logit[end_i]
                    }
                    answers.append(answer)
                    
        preds_squad.append(
            {"id": data['id'], "prediction_text": max(answers, key=lambda x: x["score"])['answer']} 
                if len(answers) > 0 else 
                {"id": data['id'], "prediction_text": ""}
        )
        
        preds_bleu.append(
            max(answers, key=lambda x: x["score"])['answer']
                if len(answers) > 0 else 
                ""
        )
        
        max_i = 0
        max_len = len(data["answers"]["text"][max_i])
        for i in range(1, len(data["answers"]['text'])):
            if len(data["answers"]["text"][i]) > max_len:
                max_len = len(data["answers"]["text"][i])
                max_i = i
        
        golds_bleu.append(data["answers"]['text'][max_i])
        
    return {
        "squad": squad_metric.compute(predictions=preds_squad, references=golds_squad),
        "bleu": bleu_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "rouge": rouge_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "semantic_similarity": util.pytorch_cos_sim(eval_answer.encode(preds_bleu, convert_to_tensor=True), eval_answer.encode(golds_bleu, convert_to_tensor=True)).mean().item() * 100
    }

# Training, evaluation and testing loop for hyperparameters

In [17]:
def output_metrics_to_file(metrics: dict, metric_type: str = None, lr: float = None, epoch: int = None):
    filename = os.path.join("/kaggle/working/" if kaggle else ".", "results", f"{model_short_name}-{dataset_short_name}-type_{metric_type}-lr_{lr}-epoch_{epoch:02}.json" if lr is not None and epoch is not None and metric_type is not None else f"{model_short_name}-{dataset_short_name}.json")
    with open(filename, "w") as f:
        json.dump(metrics, f, indent=4)
        
def save_model(trainer: Trainer, lr: float, epoch: int):
    trainer.save_model(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))
    
def load_model(lr: float, epoch: int) -> AutoModelForQuestionAnswering:
    return AutoModelForQuestionAnswering.from_pretrained(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))

## Get maximum answer length within 2 standard deviations from the mean of the training dataset

In [18]:
# mean_answer_length = np.mean([len(a["text"][0]) for a in ds_train["answers"]])
# std_dev_answer_length = np.std([len(a["text"][0]) for a in ds_train["answers"]])

# two_std_devs_above = mean_answer_length + 2 * std_dev_answer_length

In [19]:
all_models_eval = None

def tet_loop(lr_list: list, epochs: int, batch_size: int) -> None:
    global all_models_eval
    all_models_eval = {"validation": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}, "test": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}}
    
    eval_answer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")
    
    for lr in lr_list:
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)

        warmup_steps = int(len(ds_tok_train) / batch_size * 1 / 10)
        args = TrainingArguments(
                output_dir="./results",
                eval_strategy="no",
                save_strategy="epoch",
                learning_rate=lr,
                num_train_epochs=1,
                weight_decay=0.01,
                per_device_train_batch_size=batch_size, 
                report_to="none",
                save_total_limit=1,
                warmup_steps=warmup_steps
            )
        
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=ds_tok_train,
            eval_dataset=ds_tok_val,
            tokenizer=tokenizer
        )
        
        for epoch in range(1, epochs + 1):
            print(f"Training with lr={lr} and epoch={epoch}")
            
            trainer.train()
            
            preds = trainer.predict(ds_tok_val)
            start_logits, end_logits = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits, end_logits, ds_tok_val, ds_val, 30, eval_answer)
            
            # output_metrics_to_file(computed_metrics, metric_type='validation', lr=lr, epoch=epoch)
            print(f"Validation for lr {lr} epoch {epoch}: ")
            # print(computed_metrics)
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            all_models_eval['validation'][lr][epoch] = computed_metrics
            
            preds = trainer.predict(ds_tok_test)
            start_logits2, end_logits2 = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits2, end_logits2, ds_tok_test, ds_test, 30, eval_answer)
            
            
            # output_metrics_to_file(computed_metrics, metric_type='test', lr=lr, epoch=epoch)
            all_models_eval['test'][lr][epoch] = computed_metrics
            

            print(f"Test for lr {lr} epoch {epoch}: ")
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            # save_model(trainer, lr, epoch)
            
        try:
            del trainer
            del model
            # device = cuda.get_current_device()
            # device.reset()
        except Exception as e:
            print(e)
            
            print(f"Finished training, validating and testing with lr={lr} and epochs={epochs}")

In [20]:
tet_loop(lr_list=[1e-4, 1e-3, 1e-5], epochs=8, batch_size=8)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at openai-community/gpt2-medium and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1127/2730880659.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Training with lr=0.0001 and epoch=1


Step,Training Loss
500,3.275200


  0%|          | 0/119 [00:00<?, ?it/s]

Validation for lr 0.0001 epoch 1: 
{'exact_match': 6.722689075630252, 'f1': 24.15319979228622}
14.756065607070923


...

Step,Training Loss
500,2.004900


  0%|          | 0/119 [00:00<?, ?it/s]

Validation for lr 1e-05 epoch 8: 
{'exact_match': 7.563025210084033, 'f1': 22.17119201829925}
15.113957226276398


  0%|          | 0/119 [00:00<?, ?it/s]

Test for lr 1e-05 epoch 8: 
{'exact_match': 9.243697478991596, 'f1': 27.96140175527224}
16.601599752902985


In [21]:
output_metrics_to_file(all_models_eval)

In [22]:
!zip results_ml_aug.zip results/*.json

/bin/bash: line 1: zip: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
